In [1]:
## vscode 

# python: 인터프리터 선택 > conda env 동일하게 선택

In [2]:
# !pip install torch torchvision

In [3]:
# imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

import torchvision
from torchvision import transforms, datasets

In [4]:
if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
elif torch.backends.mps.is_available():
    DEVICE = torch.device('mps')
else:
    DEVICE = torch.device('cpu')

print("Using PyTorch version:", torch.__version__,' Device:', DEVICE)

BATCH_SIZE = 32
EPOCHS = 10

Using PyTorch version: 2.3.1  Device: mps


In [5]:
# transforms
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))])

# datasets
train_set = datasets.FashionMNIST('./data',
    download=True,
    train=True,
    transform=transform)
test_set = datasets.FashionMNIST('./data',
    download=True,
    train=False,
    transform=transform)

# dataloaders
train_loader = torch.utils.data.DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False)

In [6]:
# classes
classes = ('T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
        'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle Boot')

In [7]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28*28, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 10)
        
    def forward(self,x):
        x = x.view(-1, 28*28)
        x = self.fc1(x)
        x = F.sigmoid(x)
        x = self.fc2(x)
        x = F.sigmoid(x)
        x = self.fc3(x)
        x = F.log_softmax(x, dim=1)
        return x

In [8]:
model = Net().to(DEVICE)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.5)
criterion = nn.CrossEntropyLoss()

print(model)

Net(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=10, bias=True)
)


In [9]:
writer = SummaryWriter('runs/fashion_mnist_experiment')

In [10]:
def visualize_samples_tensorboard(loader, classes, samples_per_class=5):
    class_dict = {i: [] for i in range(len(classes))}
    
    for img, label in loader:
        for i in range(len(img)):
            if len(class_dict[label[i].item()]) < samples_per_class:
                class_dict[label[i].item()].append(img[i])
        if all(len(v) == samples_per_class for v in class_dict.values()):
            break
    
    for label, images in class_dict.items():
        grid = torchvision.utils.make_grid(images, nrow=samples_per_class, normalize=True)
        writer.add_image(f'{classes[label]} samples', grid, 0)

visualize_samples_tensorboard(train_loader, classes)

In [11]:
# 훈련 함수
def train(model, device, train_loader, optimizer, criterion, epoch, writer):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(output.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_accuracy = 100. * correct / total

    print(f'Train Epoch: {epoch} Loss: {epoch_loss:.4f} Accuracy: {epoch_accuracy:.2f}%')
    writer.add_scalar('Loss/train', epoch_loss, epoch)
    writer.add_scalar('Accuracy/train', epoch_accuracy, epoch)

# 테스트 함수
def test(model, device, test_loader, criterion, epoch, writer):
    model.eval()
    test_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)
            test_loss += loss.item()
            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / total

    print(f'Test Epoch: {epoch} Loss: {test_loss:.4f} Accuracy: {test_accuracy:.2f}%')
    writer.add_scalar('Loss/test', test_loss, epoch)
    writer.add_scalar('Accuracy/test', test_accuracy, epoch)

# 훈련 및 테스트 실행
for epoch in range(1, EPOCHS + 1):
    train(model, DEVICE, train_loader, optimizer, criterion, epoch, writer)
    test(model, DEVICE, test_loader, criterion, epoch, writer)

Train Epoch: 1 Loss: 0.0518 Accuracy: 42.34%
Test Epoch: 1 Loss: 0.0336 Accuracy: 60.99%
Train Epoch: 2 Loss: 0.0274 Accuracy: 68.44%
Test Epoch: 2 Loss: 0.0233 Accuracy: 72.16%
Train Epoch: 3 Loss: 0.0210 Accuracy: 74.74%
Test Epoch: 3 Loss: 0.0203 Accuracy: 75.59%
Train Epoch: 4 Loss: 0.0189 Accuracy: 77.41%
Test Epoch: 4 Loss: 0.0187 Accuracy: 78.22%
Train Epoch: 5 Loss: 0.0175 Accuracy: 79.64%
Test Epoch: 5 Loss: 0.0176 Accuracy: 79.85%
Train Epoch: 6 Loss: 0.0165 Accuracy: 81.19%
Test Epoch: 6 Loss: 0.0169 Accuracy: 80.30%
Train Epoch: 7 Loss: 0.0157 Accuracy: 82.06%
Test Epoch: 7 Loss: 0.0164 Accuracy: 81.39%
Train Epoch: 8 Loss: 0.0151 Accuracy: 82.87%
Test Epoch: 8 Loss: 0.0157 Accuracy: 82.08%
Train Epoch: 9 Loss: 0.0146 Accuracy: 83.38%
Test Epoch: 9 Loss: 0.0153 Accuracy: 82.55%
Train Epoch: 10 Loss: 0.0142 Accuracy: 83.90%
Test Epoch: 10 Loss: 0.0151 Accuracy: 82.85%


In [12]:
writer.close()